# Write my first blog  for AirBnB  Boston and Seattle Data Set

This notebook explores the AirBnB Boston and Seattle data set. Datas can be found in Kaggle: https://www.kaggle.com/airbnb/seattle, https://www.kaggle.com/airbnb/boston

The CRISP-DM structure used is:

- Business Understanding
- Data Understanding
- Prepare Data
- Data Modeling
- Evaluation


## Business Understanding
We can answer two three interesting question from the above dataset.

- When are the peak season to visit Seattle and Boston

-  How did some features (neighborhood, no. of bedrooms, etc) affect the price?

- Using Machine Learning, which features are more important related to pricing?

## Data Understanding


In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

In both cities we have three dataset:

- Listings, including full descriptions and average review score
- Reviews, including unique id for each reviewer and detailed comments
- Calendar, including listing id and the price and availability for that day

For the next, I will use two of them: listings and calendar


In [ ]:
seatlle_list = pd.read_csv('./datas/seattle/listings.csv')
seatlle_cal = pd.read_csv('./datas/seattle/calendar.csv')
boston